SIMPLE GEN Ai APP using langchain

In [2]:
import os
from dotenv import load_dotenv
load_dotenv()

os.environ['OPENAI_API_KEY']=os.getenv("OPENAI_API_KEY")
## Langsmith Tracking
os.environ["LANGCHAIN_API_KEY"]=os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"]="true"
os.environ["LANGCHAIN_PROJECT"]=os.getenv("LANGCHAIN_PROJECT")

In [3]:
# Data Ingestion from a website that we need to scrape the data from
from langchain_community.document_loaders import WebBaseLoader
loader=WebBaseLoader("https://python.langchain.com/docs/tutorials/chatbot/")
loader

In [ ]:
docs = loader.load()
docs

[Document(metadata={'source': 'https://python.langchain.com/docs/tutorials/chatbot/', 'title': 'Build a Chatbot | \uf8ffü¶úÔ∏è\uf8ffüîó LangChain', 'description': 'This tutorial previously used the RunnableWithMessageHistory abstraction. You can access that version of the documentation in the v0.2 docs.', 'language': 'en'}, page_content='\n\n\n\n\nBuild a Chatbot | \uf8ffü¶úÔ∏è\uf8ffüîó LangChain\n\n\n\n\n\n\nSkip to main contentJoin us at  Interrupt: The Agent AI Conference by LangChain on May 13 & 14 in San Francisco!IntegrationsAPI ReferenceMoreContributingPeopleError referenceLangSmithLangGraphLangChain HubLangChain JS/TSv0.3v0.3v0.2v0.1\uf8ffüí¨SearchIntroductionTutorialsBuild a Question Answering application over a Graph DatabaseTutorialsBuild a simple LLM application with chat models and prompt templatesBuild a ChatbotBuild a Retrieval Augmented Generation (RAG) App: Part 2Build an Extraction ChainBuild an AgentTaggingBuild a Retrieval Augmented Generation (RAG) App: Part 1Build

In [ ]:
### Divide the data into chunks after getting it from source
### Load Data--> Docs-->Divide our Documents into chunks documents-->text-->vectors-->Vector Embeddings--->Vector Store DB

from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)
documents = text_splitter.split_documents(docs)
documents

[Document(metadata={'source': 'https://python.langchain.com/docs/tutorials/chatbot/', 'title': 'Build a Chatbot | \uf8ffü¶úÔ∏è\uf8ffüîó LangChain', 'description': 'This tutorial previously used the RunnableWithMessageHistory abstraction. You can access that version of the documentation in the v0.2 docs.', 'language': 'en'}, page_content='Build a Chatbot | \uf8ffü¶úÔ∏è\uf8ffüîó LangChain'),
 Document(metadata={'source': 'https://python.langchain.com/docs/tutorials/chatbot/', 'title': 'Build a Chatbot | \uf8ffü¶úÔ∏è\uf8ffüîó LangChain', 'description': 'This tutorial previously used the RunnableWithMessageHistory abstraction. You can access that version of the documentation in the v0.2 docs.', 'language': 'en'}, page_content='Skip to main contentJoin us at  Interrupt: The Agent AI Conference by LangChain on May 13 & 14 in San Francisco!IntegrationsAPI ReferenceMoreContributingPeopleError referenceLangSmithLangGraphLangChain HubLangChain JS/TSv0.3v0.3v0.2v0.1\uf8ffüí¨SearchIntroductionTuto

In [6]:
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
embeddings = OpenAIEmbeddings()
vectorstoredb = FAISS.from_documents(documents,embeddings)
vectorstoredb

In [7]:
## Query From a vector db
query = "LangSmith Message persistence"
result = vectorstoredb.similarity_search(query)
result[0].page_content

"And now we can see that we get a good response!\nThis is the basic idea underpinning a chatbot's ability to interact conversationally.\nSo how do we best implement this?\nMessage persistence‚Äã\nLangGraph implements a built-in persistence layer, making it ideal for chat applications that support multiple conversational turns.\nWrapping our chat model in a minimal LangGraph application allows us to automatically persist the message history, simplifying the development of multi-turn applications.\nLangGraph comes with a simple in-memory checkpointer, which we use below. See its documentation for more detail, including how to use different persistence backends (e.g., SQLite or Postgres)."

In [8]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model="gpt-4o")

In [9]:
## retrieval chain, document chain

from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

## context here is the information I will be giving w.r.t. info of document or text to llm
prompt = ChatPromptTemplate.from_template(
    """
        Answer the following question based only on the provided context:
            <context>
                {context}
            </context>


    """
)

document_chain = create_stuff_documents_chain(llm,prompt)
document_chain

RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableLambda(format_docs)
}), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
| ChatPromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='\n        Answer the following question based only on the provided context:\n            <context>\n                {context}\n            </context>\n\n\n    '), additional_kwargs={})])
| ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x000001AE10E64C50>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x000001AE10D01790>, root_client=<openai.OpenAI object at 0x000001AE109219A0>, root_async_client=<openai.AsyncOpenAI object at 0x000001AE10E64CB0>, model_name='gpt-4o', model_kwargs={}, openai_api_key=SecretStr(

In [11]:
from langchain_core.documents import Document
document_chain.invoke({
    "input":"LangSmith Message persistence",
    "context":[Document(page_content="LangSmith Message persistence")]
})

"LangSmith Message persistence refers to the ability of LangSmith to store or maintain messages over time. However, the provided context does not include specific details about how this persistence is implemented or managed within LangSmith. If you have further details or a specific aspect you're interested in, please provide more information."

In [12]:
### Input => Retriever => vectorstoredb
from langchain.chains import create_retrieval_chain
vectorstoredb

retriever = vectorstoredb.as_retriever()
retrieval_chain = create_retrieval_chain(retriever,document_chain)

In [13]:
retrieval_chain

RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableBinding(bound=RunnableLambda(lambda x: x['input'])
           | VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000001AE31702E70>, search_kwargs={}), kwargs={}, config={'run_name': 'retrieve_documents'}, config_factories=[])
})
| RunnableAssign(mapper={
    answer: RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
              context: RunnableLambda(format_docs)
            }), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
            | ChatPromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='\n        Answer the following question based only on the provided context:\n            <context>\n                {context}\n            </context>

In [14]:
response=retrieval_chain.invoke({"input":"LangSmith Message persistence"})
response['answer']

"Based on the provided context, the document discusses the implementation of a chatbot using LangGraph and focuses on features like message persistence and streaming to enhance user experience. The chatbot is designed to maintain conversational memory over multiple turns by utilizing LangGraph's persistence layer. It also mentions improving user experience through token streaming, allowing users to see progress in real-time as the language model generates responses.\n\nAdditionally, prompt templates are introduced as a way to structure raw user input into a format that a language model can process effectively. This involves adding system messages with custom instructions and considering additional input types.\n\nOverall, the main focus is on designing an LLM-powered chatbot that can remember previous interactions and improve with the use of structured input templates."

In [15]:
response

{'input': 'LangSmith Message persistence',
 'context': [Document(id='12b5eb04-a768-47de-a717-df75a7eae613', metadata={'source': 'https://python.langchain.com/docs/tutorials/chatbot/', 'title': 'Build a Chatbot | \uf8ffü¶úÔ∏è\uf8ffüîó LangChain', 'description': 'This tutorial previously used the RunnableWithMessageHistory abstraction. You can access that version of the documentation in the v0.2 docs.', 'language': 'en'}, page_content="And now we can see that we get a good response!\nThis is the basic idea underpinning a chatbot's ability to interact conversationally.\nSo how do we best implement this?\nMessage persistence‚Äã\nLangGraph implements a built-in persistence layer, making it ideal for chat applications that support multiple conversational turns.\nWrapping our chat model in a minimal LangGraph application allows us to automatically persist the message history, simplifying the development of multi-turn applications.\nLangGraph comes with a simple in-memory checkpointer, which w

In [16]:
response['context']

[Document(id='12b5eb04-a768-47de-a717-df75a7eae613', metadata={'source': 'https://python.langchain.com/docs/tutorials/chatbot/', 'title': 'Build a Chatbot | \uf8ffü¶úÔ∏è\uf8ffüîó LangChain', 'description': 'This tutorial previously used the RunnableWithMessageHistory abstraction. You can access that version of the documentation in the v0.2 docs.', 'language': 'en'}, page_content="And now we can see that we get a good response!\nThis is the basic idea underpinning a chatbot's ability to interact conversationally.\nSo how do we best implement this?\nMessage persistence‚Äã\nLangGraph implements a built-in persistence layer, making it ideal for chat applications that support multiple conversational turns.\nWrapping our chat model in a minimal LangGraph application allows us to automatically persist the message history, simplifying the development of multi-turn applications.\nLangGraph comes with a simple in-memory checkpointer, which we use below. See its documentation for more detail, inc